1.3.2. Phase 2: Policy Document Indexing for RAG

Index benefits policy PDFs for Retrieval-Augmented Generation (RAG) to enable
efficient policy retrieval.

Text Extraction: Use PyPDF2 to extract text from policy PDFs, removing noise
(e.g., page numbers).

Text Preprocessing: Segment text into chunks (e.g., 200 words per chunk).

Vector Store Creation: Build a Chroma vector store using embeddings from 
sentence-transformers or OpenAI.

Retrieval Testing: Test retrieval with 5 HR policy queries (e.g., “What is the
eligibility for Tuition Reimbursement?”) and evaluate chunk relevance.

In [2]:
!pip install PyPDF2

In [105]:
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter,CharacterTextSplitter
import os
import pandas as pd

directory_path = './assets/benefits/'  # Replace with the actual directory path

pdf_rows = []
text_splitter = CharacterTextSplitter(separator='\n\n',chunk_size=200,chunk_overlap=0)

for filename in os.listdir(directory_path):
    if filename.endswith('.pdf'):
        full_path = os.path.join(directory_path, filename)
        print(f"Found PDF: {full_path}")
        reader = PdfReader(full_path)
        total_pages = len(reader.pages)
        print(f"Total pages: {total_pages}")
        for i in range(total_pages):
            chunk_num = 0
            page_text = reader.pages[i].extract_text()
            page_text = page_text.replace("Introduction", '')
            chunks = text_splitter.split_text(page_text)
            for c in chunks:
                pdf_rows.append({'policy': filename, 'page': i, 'chunk': chunk_num,'text': c})
                chunk_num+=1
policy_pdfs = pd.DataFrame(pdf_rows)

Found PDF: ./assets/benefits/gym-policy.pdf
Total pages: 7
Found PDF: ./assets/benefits/life-insurance-policy.pdf
Total pages: 8
Found PDF: ./assets/benefits/childcare-policy.pdf
Total pages: 8
Found PDF: ./assets/benefits/401k-retirement-policy.pdf
Total pages: 9
Found PDF: ./assets/benefits/work-from-home-policy.pdf
Total pages: 7
Found PDF: ./assets/benefits/vacation-policy.pdf
Total pages: 7
Found PDF: ./assets/benefits/health-insurance-policy.pdf
Total pages: 7
Found PDF: ./assets/benefits/tuition-reimbursement-policy.pdf
Total pages: 8


In [ ]:
!pip install -qU "langchain-chroma"
!pip install -qU langchain-openai

In [122]:
import getpass
import os
from langchain_openai import OpenAIEmbeddings

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from uuid import uuid4

vector_store = Chroma(collection_name="policy_embeddings",embedding_function=embeddings,
    persist_directory="./assets/chroma")

docs = []
idx = 0
#vector_store.delete(ids=uuids[:-1])
for row in policy_pdfs.itertuples():
    doc = Document(page_content=row.text, metadata={'policy': row.policy, 'page': row.page}, id=idx)
    docs.append(doc)
    idx+=1
uuids = [str(uuid4()) for _ in range(len(docs))]
vector_store.add_documents(documents=docs, ids=uuids)

In [ ]:
results = vector_store.similarity_search_with_score(
    "can i join the gym?", k=2
)
for res, score in results:
    print(f"* [Similarity ={score:3f}] {res.page_content} [{res.metadata}]")